# 데이터 전처리(Preprocessing)

spacy 라이브러리: 문장의 토큰화, 태깅 등의 전처리 기능을 위한 라이브러리

영어와 독일어 전처리 모듈 설치

In [1]:
%%capture
!python -m spacy download en
!python -m spacy download de

In [2]:
import spacy

In [3]:
spacy_en = spacy.load('en_core_web_sm')
spacy_de = spacy.load('de_core_news_sm')

In [4]:
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
  print(f"인덱스 {i} : {token.text}")

인덱스 0 : I
인덱스 1 : am
인덱스 2 : a
인덱스 3 : graduate
인덱스 4 : student
인덱스 5 : .


토큰화 함수 정의

In [5]:
# 독일어 문장을 토큰화 하는 함수
def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]

# 영어 문장을 토큰화 하는 함수
def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]

In [14]:
import torch
print(torch.__version__)

2.1.0+cu121


In [17]:
!pip install torchtext==0.16.0

In [20]:
# 최신 버전의 torchtext는 Field 지원안함.
'''
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize='spacy', tokenizer_language='en', init_token='', eos_token='', lower=True)
TRG = Field(tokenize='spacy', tokenizer_language='de', init_token='', eos_token='', lower=True)

train_data, valid_data, test_data = Multi30k.splits(exts=('.en', '.de'), fields=(SRC, TRG))
'''

"\nfrom torchtext.datasets import Multi30k\nfrom torchtext.data import Field, BucketIterator\n\nSRC = Field(tokenize='spacy', tokenizer_language='en', init_token='', eos_token='', lower=True)\nTRG = Field(tokenize='spacy', tokenizer_language='de', init_token='', eos_token='', lower=True)\n\ntrain_data, valid_data, test_data = Multi30k.splits(exts=('.en', '.de'), fields=(SRC, TRG))\n"